# Keras

In [ ]:
from keras.models import Sequential

model = Sequential()

In [ ]:
from keras.layers import Dense
model.add(Dense(units=64, activation='relu',input_dim=100))
model.add(Dense(units=10,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='sgd',
             metrics=['accuracy'])

In [ ]:
simple= Sequential()
simple.add(Dense(32,activation='relu',input_dim=100))
simple.add(Dense(1,activation='sigmoid'))
simple.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
import numpy as np
data = np.random.random((1000,100))
labels = np.random.randint(2,size=(1000,1))
simple.fit(data,labels,epochs=10,batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
results=simple.predict(data)
results=np.where(results>0.5,1,0)
plt.scatter(np.arange(50),results[0:50],c='r',marker='o')
plt.scatter(np.arange(50),labels[0:50],c='b',marker='x')
plt.show()
# print(results)

## MLP for multi-class softmax classification

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
import numpy as np
x_train= np.random.random((1000,20))
y_train= keras.utils.to_categorical(np.random.randint(10,size=(1000,1)),
                                   num_classes=10)
x_test= np.random.random((100,20))
y_test= keras.utils.to_categorical(np.random.randint(10,size=(100,1)),
                                   num_classes=10)
model = Sequential()
model.add(Dense(64,activation='relu',input_dim=20))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(10,activation='softmax'))

sgd=keras.optimizers.SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,
             metrics=['accuracy'])
model.fit(x_train,y_train,
         epochs=20,batch_size=128)
score=model.evaluate(x_test,y_test,batch_size=128)

In [ ]:
print(score)

## VGG like convnet

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=10, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=10)

In [ ]:
print(score)

## Functional API

### Densely-connected network

In [1]:
from keras.layers import Input,Dense
from keras.models import Model

inputs = Input(shape=(784,))
x = Dense(64,activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=inputs,outputs=predictions)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',
             metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers import TimeDistributed
input_sequences = Input(shape=(20,784))
processed_sequences=TimeDistributed(model)(input_sequences)

In [3]:
print(processed_sequences)

Tensor("time_distributed_1/Reshape_1:0", shape=(?, 20, 10), dtype=float32)


In [5]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
main_input=Input(shape=(100,), dtype='int32', name='main_input')
x= Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out= LSTM(32)(x)

In [6]:
auxiliary_output=Dense(1,activation='sigmoid',name='aux_output')(lstm_out)

In [7]:
import keras
auxiliary_input= Input(shape=(5,),name='aux_input')
x=keras.layers.concatenate([lstm_out,auxiliary_input])
x= Dense(64,activation='relu')(x)
x= Dense(64,activation='relu')(x)
x= Dense(64,activation='relu')(x)
main_output=Dense(1,activation='sigmoid',name='main_output')(x)


In [8]:
model=Model(inputs=[main_input,auxiliary_input],outputs=[main_output,auxiliary_output])

In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',
             loss_weights=[1.,0.2])

### Shared Layers

In [9]:
import keras
from keras.layers import Input,LSTM,Dense
from keras.models import Model
tweet_a= Input(shape=(280,256))
tweet_b= Input(shape=(280,256))

In [ ]:
shared_lstm=LSTM(64)
encoded_a=shared_lstm(tweet_a)
encoded_b=shared_lstm(tweet_b)

merged_vector = keras.layers.concatenate([encoded_a,encoded_b])

predictions=Dense(1,activation='sigmoid')(merged_vector)

model=Model(inputs=[tweet_a,tweet_b],outputs=predictions)

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])